In [1]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random as rn
from skimage.feature import hog

In [2]:
veh_img_arr=glob.glob('./ReTrucks/*.jpg')

veh_img_orig=[]
for imgPath in veh_img_arr:
    img=cv2.imread(imgPath)
    rgbImg=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    veh_img_orig.append(rgbImg)

print('Reading of Vehicle Images Done')
print('No. of Vehicle Images loaded='+str(len(veh_img_arr)))

non_veh_orig=[]
non_veh_arr=glob.glob('./non-vehicles/*/*.png')
for imgPath in non_veh_arr:
    img=cv2.imread(imgPath)
    rgbImg=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    non_veh_orig.append(rgbImg)



print('Reading of Non-Vehicle Images Done')
print('No. of Non-Vehicle Images loaded='+str(len(non_veh_arr)))

Reading of Vehicle Images Done
No. of Vehicle Images loaded=8536
Reading of Non-Vehicle Images Done
No. of Non-Vehicle Images loaded=8968


In [3]:
def GetFeaturesFromHog(image,orient,cellsPerBlock,pixelsPerCell, visualise= False, feature_vector_flag=True):
    if(visualise==True):
        hog_features, hog_image = hog(image, orientations=orient,
                          pixels_per_cell=(pixelsPerCell, pixelsPerCell), 
                          cells_per_block=(cellsPerBlock, cellsPerBlock), 
                          visualise=True, feature_vector=feature_vector_flag)
        return hog_features, hog_image
    else:
        hog_features = hog(image, orientations=orient,
                          pixels_per_cell=(pixelsPerCell, pixelsPerCell), 
                          cells_per_block=(cellsPerBlock, cellsPerBlock), 
                          visualise=False, feature_vector=feature_vector_flag)
        return hog_features

In [4]:
def ExtractFeatures(images,orientation,cellsPerBlock,pixelsPerCell,convertColorspace=False):
    featureList=[]
    imageList=[]
    for image in images:
        if(convertColorspace==True):
            image=cv2.cvtColor(image, cv2.COLOR_RGB2YCR_CB)
        local_features0=GetFeaturesFromHog(image[:,:,0],orientation,cellsPerBlock,pixelsPerCell,False,True)
        local_features1=GetFeaturesFromHog(image[:,:,1],orientation,cellsPerBlock,pixelsPerCell,False,True)
        local_features2=GetFeaturesFromHog(image[:,:,2],orientation,cellsPerBlock,pixelsPerCell,False,True)
        x=np.hstack((local_features0,local_features1,local_features2))
        featureList.append(x)
    return featureList

In [5]:
%%time

orientations=9
cellsPerBlock=2
pixelsPerCell=16
convertColorspace=True
vehicleFeatures=ExtractFeatures(veh_img_orig,orientations,cellsPerBlock,pixelsPerCell,convertColorspace)
nonVehicleFeatures=ExtractFeatures(non_veh_orig,orientations,cellsPerBlock,pixelsPerCell,convertColorspace)

C:\Users\dell\Anaconda2\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
C:\Users\dell\Anaconda2\lib\site-packages\skimage\feature\_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


Wall time: 48.3 s


In [6]:
featuresList=np.vstack([vehicleFeatures,nonVehicleFeatures])
print('Shape of features list is ', str((featuresList.shape)))
labelList= np.concatenate([np.ones(len(vehicleFeatures)), np.zeros(len(nonVehicleFeatures))])
print('Shape of label list is ', str((labelList.shape)))

('Shape of features list is ', '(17504L, 972L)')
('Shape of label list is ', '(17504L,)')


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(featuresList, labelList, test_size=0.2, shuffle=True)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

In [9]:
%%time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')
classifier2=AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators = 50, learning_rate = 1)
classifier2.fit(X_train, Y_train)
print("Accuracy of Adaboost is ", classifier2.score(X_test,Y_test))

('Accuracy of Adaboost is ', 0.988860325621251)
Wall time: 4min 49s


In [10]:
%%time

from sklearn.svm import LinearSVC
classifier1=LinearSVC()
classifier1.fit(X_train,Y_train)
print("Accuracy of SVC is ", classifier1.score(X_test,Y_test))

('Accuracy of SVC is ', 0.9728648957440731)
Wall time: 1.31 s


In [11]:
%%time
import xgboost as xgb
classifier3=xgb.XGBClassifier()
classifier3.fit(X_train, Y_train)
print("Accuracy of XGBoost is ", classifier3.score(X_test,Y_test))

('Accuracy of XGBoost is ', 0.9848614681519566)
Wall time: 1min 48s


C:\Users\dell\Anaconda2\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
